In [9]:
# Import necessary packages
from astroML.correlation import two_point_angular


%pylab inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import InterpolatedUnivariateSpline






fontsize = 8
figsize = (3,3)
dpi = 250


# Configure parameters
plt.style.use('default')
plt.rcParams.update({'font.size': fontsize, 'figure.figsize': figsize, 'figure.dpi':dpi})


# Default tick label size
plt.rcParams['xtick.labelsize'] = fontsize
plt.rcParams['ytick.labelsize'] = fontsize
plt.rcParams['text.usetex'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2

plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['axes.linewidth'] = 2


Populating the interactive namespace from numpy and matplotlib


/Users/bbonine/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['figsize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [15]:
# Read in data 

# Read in AGN table:
path = '/Users/bbonine/ou/research/corr_func/data/'
# Remote version: cat = "/home/bonine/donnajean/research/agn_corr/data/agntable_total.txt"
cat = path + 'agntable_total.txt'
field = np.loadtxt(cat, dtype = str,delimiter = None, skiprows = 1, usecols=(15) , unpack = True)
#Get rid of any duplicates:
field_list = np.unique(field)
x,y = np.loadtxt(cat, delimiter = None, skiprows = 1, usecols=(16,17) , unpack = True)


In [13]:
# Read in the flux limit file: 
#lim = '/home/bonine/donnajean/research/agn_corr/data/fluxlimit.txt'
lim = path + 'fluxlimit.txt'
exp, fluxlim = np.loadtxt(lim,skiprows = 1, unpack = True)
exp = np.power(10,exp) #exposure time in log units; convert


In [14]:
# Interpolate the flux values:
func1 = InterpolatedUnivariateSpline(exp,fluxlim) 
xnew = np.linspace(0,10**8, num = 10**7, endpoint = True)

In [16]:
field_list

array(['grb041219a', 'grb041219b', 'grb041223', 'grb050117', 'grb050124',
       'grb050126', 'grb050128', 'grb050209', 'grb050215b', 'grb050219a',
       'grb050219b', 'grb050223', 'grb050306', 'grb050315', 'grb050318',
       'grb050319', 'grb050326', 'grb050401', 'grb050406', 'grb050408',
       'grb050410', 'grb050412', 'grb050416a', 'grb050416b', 'grb050421',
       'grb050422', 'grb050502a', 'grb050502b', 'grb050504', 'grb050505',
       'grb050509a', 'grb050509b', 'grb050509c', 'grb050520', 'grb050522',
       'grb050525a', 'grb050528', 'grb050603', 'grb050607', 'grb050626',
       'grb050701', 'grb050709', 'grb050712', 'grb050713a', 'grb050713b',
       'grb050714a', 'grb050714b', 'grb050716', 'grb050717', 'grb050721',
       'grb050724', 'grb050726', 'grb050730', 'grb050801', 'grb050802',
       'grb050803', 'grb050813', 'grb050814', 'grb050815', 'grb050819',
       'grb050820a', 'grb050820b', 'grb050822', 'grb050824', 'grb050826',
       'grb050827', 'grb050904', 'grb050906',

In [40]:
# Read in a particular exposure map
num = 117
here = np.where(field == field_list[num])
# Extract source positions in this field:
data_x = x[here]
data_y = y[here]

print(field_list[num])

grb060202


In [41]:
# Read in exposure map with astropy
expmap = path + field_list[num] +'/expo.fits'
hdu_list = fits.open(expmap)
image_data = hdu_list[0].data
hdu_list.close()
exp_map_1d =  image_data.ravel() #Conver exposure map to 1D array for later

# Restrict to fields with more than one AGN (necessary for correlation calculation):

# Save reference pixel value for later
ref_flux =  image_data[500,500]

# Use the interpolated function to extract flux limit based off reference flux
flux_lim = func1(ref_flux)

# Find the flux limit for each pixel:
fluxlimit = np.zeros(len(exp_map_1d))
for j in range(0,len(fluxlimit)):
    fluxlimit[j] = func1(exp_map_1d[j])
    
fluxlimit_1d = np.asarray(fluxlimit) #convert to numpy array
fluxlimit_2d = np.reshape(fluxlimit_1d,(-1,len(image_data[0])))
    

In [42]:
fluxlimit_1d

array([7.67331525e-12, 7.67331525e-12, 7.67331525e-12, ...,
       7.67331525e-12, 7.67331525e-12, 7.67331525e-12])

In [49]:
bins = np.linspace(0,1000,8)
two_point_angular(data_x,data_y, bins, method = 'landy-szalay')

array([        nan,         nan,  3.        ,  0.30948122, -0.22190202,
       -0.35947302, -0.34673367])

In [48]:
bins

array([   0.        ,  157.14285714,  314.28571429,  471.42857143,
        628.57142857,  785.71428571,  942.85714286, 1100.        ])